In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
tapo_username = os.getenv("TAPO_USERNAME")
tapo_password = os.getenv("TAPO_PASSWORD")
pushover_user_group = os.getenv("PUSHOVER_USER_GROUP_WOERIS")

In [2]:
pushover_user_group

'gh9maxhrd44jxy8qx7ka1jc2cfiykg'

In [3]:

from datetime import datetime
print(datetime.now())

2025-07-29 20:16:23.000153


In [4]:
from utils import send_pushover_notification_new
send_pushover_notification_new(pushover_user_group, "Test with new send method")

In [4]:
import http.client, urllib
conn = http.client.HTTPSConnection("api.pushover.net:443")

pushover_api_token = os.getenv("PUSHOVER_TAPO_API_TOKEN")
conn.request("POST", "/1/messages.json",
  urllib.parse.urlencode({
    "token": pushover_api_token,
    "user": pushover_user_group,
    "message": "hello world",
  }), { "Content-type": "application/x-www-form-urlencoded" })
conn.getresponse()

In [8]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "None"
url = "http://192.168.178.114:8088"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [9]:
bucket="test_bucket"

write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
    point = (
        Point("measurement1")
        .tag("tagname1", "tagvalue1")
        .field("field1", value)
    )
    write_api.write(bucket=bucket, org="None", record=point)
    time.sleep(1) # separate points by 1 second

In [11]:
query_api = write_client.query_api()

query = """from(bucket: "test_bucket")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="None")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 28, 25, 376630, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 12, 13, 15, 26, 20, 935153, tzinfo=datetime.timezone.utc), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 28, 25, 376630, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 12, 13, 15, 26, 23, 421039, tzinfo=datetime.timezone.utc), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_

In [12]:
query_api = write_client.query_api()

query = """from(bucket: "test_bucket")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")
    |> mean()"""
tables = query_api.query(query, org="None")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 19, 1, 732013, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 29, 1, 732013, tzinfo=datetime.timezone.utc), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}


In [13]:
tables

[<FluxTable: 8 columns, 1 records>]

In [1]:
import asyncio
import os
from tapo import ApiClient
from dotenv import load_dotenv

from utils import monitor_power_and_notify


load_dotenv()
tapo_username = os.getenv("TAPO_USERNAME")
tapo_password = os.getenv("TAPO_PASSWORD")
pushover_user_group = os.getenv("PUSHOVER_USER_GROUP_WOERIS")
wasching_machine_ip_address = os.getenv("WASCHING_MACHINE_IP_ADDRESS")

client = ApiClient(tapo_username, tapo_password)


In [2]:
device_wasching_machine = await client.p110(wasching_machine_ip_address)

In [3]:
device_wasching_machine

In [ ]:
await monitor_power_and_notify(device=device_wasching_machine, user=pushover_user_group, message="Die Wäsche ist fertig, Tapsi! 🧺🐶")

Current power: 0W


In [7]:
from datetime import datetime
print(datetime.now())

2025-01-01 20:30:56.315561


In [22]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

# InfluxDB connection parameters
influx_url = "http://192.168.178.114:8088"
influx_token = os.environ.get("INFLUXDB_TOKEN")
influx_org = "None"
influx_bucket = "power_consumption"

# Time range for data export (adjust as needed)
start_time = datetime.now() - timedelta(days=49)  # Last 30 days
end_time = datetime.now()  # Up to yesterday

# Query to get all power consumption data
query = f'''
from(bucket: "{influx_bucket}")
  |> range(start: {start_time.strftime("%Y-%m-%dT%H:%M:%SZ")}, stop: {end_time.strftime("%Y-%m-%dT%H:%M:%SZ")})
  |> filter(fn: (r) => r._measurement == "power_consumption")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# query = f'''
# from(bucket: "{influx_bucket}")
#   |> range(start: 0)
#   |> filter(fn: (r) => r._measurement == "power_consumption")
#   |> sort(columns: ["_time"], desc: false)
#   |> limit(n:1)
# '''

# Connect to InfluxDB and execute query
with InfluxDBClient(url=influx_url, token=influx_token, org=influx_org) as client:
    query_api = client.query_api()
    try:
        # Set a timeout for the query to prevent hanging
        result = query_api.query_data_frame(query)  # 60 seconds timeout in milliseconds
    except Exception as e:
        print(f"Error querying InfluxDB: {e}")
        # Create empty DataFrame with expected columns if query fails
        result = pd.DataFrame(columns=['_time', 'device', 'power'])
    
    if isinstance(result, list):
        # If multiple DataFrames are returned, concatenate them
        df = pd.concat(result)
    else:
        df = result
    
    # Clean up the DataFrame
    if not df.empty:
        # Select and rename relevant columns
        df = df[['_time', 'device', 'power']]
        df.rename(columns={'_time': 'timestamp', 'device': 'device_name', 'power': 'power_watts'}, inplace=True)
        
        # Export to CSV
        # csv_filename = f"power_consumption_export_{datetime.now().strftime('%Y%m%d')}.csv"
        # df.to_csv(csv_filename, index=False)
        # print(f"Data exported to {csv_filename}")
        
        # Optionally export to Parquet
        parquet_filename = f"power_consumption_export_{datetime.now().strftime('%Y%m%d')}.parquet"
        df.to_parquet(parquet_filename, index=False)
        print(f"Data exported to {parquet_filename}")
    else:
        print("No data found for the specified time range")

Data exported to power_consumption_export_20250424.parquet


/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_71877/828729857.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'_time': 'timestamp', 'device': 'device_name', 'power': 'power_watts'}, inplace=True)


In [4]:
print(influx_token)

If6bH_EIYLSOXw802HsWc7wbBZKIjB3NWc11lEslNoJwoZzSo804b0oYOdoE4a0lSNZtHfJHNn34l8546gdzMQ==


In [23]:
df_49_0 = df.copy()

In [24]:
df = pd.concat([df_130_90, df_89_50, df_49_0], ignore_index=True)

In [25]:
df

,timestamp,device_name,power_watts
0,2024-12-15 22:52:04.072673+00:00,bedroom,3
1,2024-12-15 22:52:38.150659+00:00,bedroom,3
2,2024-12-15 22:53:10.560969+00:00,bedroom,3
3,2024-12-15 22:53:44.858782+00:00,bedroom,3
4,2024-12-15 22:54:18.295927+00:00,bedroom,2
...,...,...,...
2967809,2025-04-24 20:52:09.185973+00:00,washing_machine,0
2967810,2025-04-24 20:52:41.950741+00:00,washing_machine,0
2967811,2025-04-24 20:53:14.129687+00:00,washing_machine,0
2967812,2025-04-24 20:53:46.367341+00:00,washing_machine,0


In [26]:
parquet_filename = f"power_consumption_export_{datetime.now().strftime('%Y%m%d')}.parquet"
df.to_parquet(parquet_filename, index=False)
print(f"Data exported to {parquet_filename}")

Data exported to power_consumption_export_20250424.parquet


In [4]:
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# Create a line plot for power consumption by device over time
fig = px.line(df, 
              x='timestamp', 
              y='power_watts', 
              color='device_name',
              title='Power Consumption Over Time by Device',
              labels={
                  'timestamp': 'Date & Time',
                  'power_watts': 'Power (Watts)',
                  'device_name': 'Device'
              })

# Improve the layout
fig.update_layout(
    legend_title_text='Device',
    xaxis_title='Date & Time',
    yaxis_title='Power (Watts)',
    height=600,
    hovermode='closest'
)

# Add range slider
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

ModuleNotFoundError: No module named 'plotly'